In [99]:
#Librerias necesarias
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import time
from datetime import datetime
import math

#Driver de chrome 
from selenium.webdriver.chrome.service import Service
chrome_driver = "chromedriver.exe"
driver_service = Service(executable_path = chrome_driver)

## EL DRIVER DEBE ESTAR EN LA MISMA CARPETA QUE EL ARCHIVO

In [100]:
def busca_coches(precio_minimo,num_coches):# Funcion que almacena los links de los coches de una página de Wallapop
    links = [] #Donde se almacenaran todos los links
    print("coches de de",precio_minimo,"a",precio_minimo+1000) #Chivato para saber por donde vamos
    url = "https://es.wallapop.com/app/search?min_sale_price="+str(precio_minimo)+"&max_sale_price="+str((precio_minimo+1000))+"&filters_source=default_filters&category_ids=100&longitude=-3.69196&latitude=40.41956"
    navegador.get(url) ##Cogemos la url
    time.sleep(3) ## Esperamos un poco a que cargue, por si acaso
    navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    boton = navegador.find_element(By.ID, "btn-load-more") ## El boton de cargar mas productos. AVECES DA ERROR PORQUE DI
    boton.click() ##Pulsamos el boton AVECES DA ERROR DEJAR EL NAVEGADOR ABIERTO COMPLETAMENTE PARA QUE NO OCURRA
    time.sleep(2) ## El boton parece que tarda bastante en cargar, por eso puse un timer de 5 segundos, seguro que se podrá bajar
        
    for i in range(round((num_coches)/40)): #Cada iteracion corresponde a 40 fichas de coches más, a partir de 80 interacciones ya comienza a ir lento en mi pc
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);") ## Hacemos scroll tantas veces como digamos en el for
        sleep(1)
        print("Cargando",i) ## Para saber que funciona
            
    navegador.page_source ## Necesitamos el soup de la página con todo el scroll
    soup = BeautifulSoup(navegador.page_source) #Soup de la pagina cargada
    time.sleep(2) #Que cargue todo
    ficha = soup.find_all("a", attrs = {"class":"ItemCardList__item"}) #Encuentra la ficha de todos los coches
        
    for i in range(len(ficha)): #Para cada ficha de coche...
        links.append(ficha[i]["href"]) #Añade links a la lista
    print(len(ficha),"Links metidos") #Chivato, nos dira cuantos links de coches ha introducido

    #Una vez tenemos todos los links ejecutamos la funcion que recopila la informacion de cada uno de ellos
    #data_pagina = sacar_datos(links) 
    return links #Devolverá los datos 

In [101]:
def sacar_datos(links): ## Función que saca los datos de cada link obtenido de la funcion saca_datos
    
    precios=[]
    marcas=[]
    modelos=[]
    year=[]
    kms=[]
    type_vehi=[]
    plazas=[]
    puertas=[]
    type_comb=[]
    potencias=[]
    type_cambio=[]
    texto=[]
    nombre_usuario=[]
    estrellas=[]
    resenas=[]
    ubicacion=[]
    num_ventas=[]
    
    for j in links: #Por cada anuncio de la página...
        print(j)
        navegador.get(j) #Entramos en la URL
        time.sleep(1)
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)
        soup = BeautifulSoup(navegador.page_source)
        error = soup.find('div', attrs = {'class': 'not-found-page_Error__container__GtFkg d-flex flex-column align-items-center'}) #Para cuando la pagina no exista
        error2 = soup.find('div', attrs = {'class': 'error-page_Error__container__qaYIn d-flex flex-column align-items-center'}) ## Para cuando la pagina de error
        if error != None: ## Si la pagina no existe el contenido del error no estará vacio
            continue ## Pasaremos a la siguiente iteracion, ya que en la página vacia no hay datos que poner       
        if error2 != None: ## Para otro problema en la pagina
            continue

            
        precio=soup.find('div', attrs={'aria-label':'Item Price'})
        if len(precio)==1:
            precios.append(precio.span.text.split()[0])
        elif len(precio)==2:
            p=precio.find('span', attrs={'class':'item-detail-price_ItemDetailPrice--standardFinanced__14D3z'})
            precios.append(p.text.split()[0])
        else:
            precios.append(np.NaN)
    # Vamos a sacar el nombre de usuario
        nombre = soup.find_all('h3')
        
        # nombre = soup.find('h3', attrs={'class':'text-truncate mb-0 item-detail-header_ItemDetailHeader__text--typoMidM__VeCLc'})
        if nombre is not None:
            nombre_usuario.append(nombre[0].text)
        else:
            nombre_usuario.append(np.NaN)

    # Vamos a sacar la lista de comentarios. El if/else es para cuando alguno nos de un problema.
        textos = soup.find("section", attrs = {"class":"item-detail_ItemDetail__description__7rXXT py-4"})
        if textos is not None:
            texto.append(textos.text)
        else:
            texto.append(np.NaN)

    # Estrellas
        stars = soup.find("span", attrs = {"class":"item-detail-header_ItemDetailHeader__text--typoLowS__9JNQi ml-1"})
        if stars != None:
            estrellas.append(stars.text)
        else:
            estrellas.append(np.NaN)

    # Reseñas 
        num_resenas = soup.find("span", attrs = {"class":"item-detail-header_ItemDetailHeader__text--typoLowSContentLow__UyhmS ml-1"})
        if num_resenas != None:
            resenas.append(num_resenas.text[1])
        else:
            resenas.append(np.NaN)
    # Nº de ventas
        num_venta = soup.find('span', attrs = {'class': 'item-detail-header_ItemDetailHeader__text--typoLowS__9JNQi mr-1'})
        if num_venta != None:
            num_ventas.append(num_venta.text.split()[0])
        else:
            num_ventas.append(np.NaN)
    # Ubicación
        ubicaciones = soup.find("a", attrs = {"class":"item-detail-location_ItemDetailLocation__link__s4oPx"})
        if ubicaciones == None: ## Si no esta ese etiqueta buscamos en la otra posible
            ubicaciones = soup.find("div", attrs = {"class":"d-flex item-detail-location_ItemDetailLocation___QiCU"})
            if ubicaciones == None: ##En casa de que tampoco hubiera ubicacion
                continue ##Pase al siguiente
        ubicacion.append(ubicaciones.text)
    # Vamos a sacar la lista de características (tipo, nº de plazas, nº de puertas, , tipo de combustible, potencia en CV, tipo de cambio).
    # El if/else es para cuando alguno nos de un problema.
        carac=soup.find_all('span', attrs={'class':"item-detail-attributes-info_AttributesInfo__measure__uZS62 mt-1"})
        if len(carac)==6:
            type_vehi.append(carac[0].text)
            plazas.append(carac[1].text.split()[0])
            puertas.append(carac[2].text.split()[0])
            type_comb.append(carac[3].text)
            potencias.append(carac[4].text.split()[0])
            type_cambio.append(carac[5].text)
        else:
            type_vehi.append(np.NaN)
            plazas.append(np.NaN)
            puertas.append(np.NaN)
            type_comb.append(np.NaN)
            potencias.append(np.NaN)
            type_cambio.append(np.NaN)
            
    # Vamos a sacar la lista de varios (marca, modelo, año y kilómetros).
    # El if/else es para cuando alguno nos de un problema.
        varios=soup.find_all('div',attrs={'class':"item-detail-car-extra-info_ItemDetailCarExtraInfo__section__n4g_P d-flex align-items-center"})
        if len(varios)==4:
            marcas.append(varios[0].find_all('span')[1].text)
            modelos.append(varios[1].find_all('span')[1].text)
            year.append(varios[2].find_all('span')[1].text)
            kms.append(varios[3].find_all('span')[1].text)

        else:
            marcas.append(np.NaN)
            modelos.append(np.NaN)
            year.append(np.NaN)
            kms.append(np.NaN)

########################################################################################################

    keys = ['Marca', 'Modelo', 'Año', 'Potencia', 'Plazas', 'Nº de puertas',
         'km', 'Tipo de vehiculo', 'Combustible', 'Cambio', 'Comentarios','Nombre usuario',
          'Puntuacion', 'Nº valoraciones', 'Nº Ventas', 'Ubicacion', 'Precio' ]
    values = [marcas, modelos, year, potencias, plazas, puertas,
           kms, type_vehi, type_comb, type_cambio, texto, nombre_usuario,
           estrellas, resenas, num_ventas, ubicacion, precios]
    dic1={}
# Usamos un bucle para asignar las claves y valores al diccionario
    for key, value in zip(keys, values):
        dic1[key] = value
 # Excepcion rara en la que a veces los diccionarios no tienen el mismo numeo de elementos         
    df=pd.DataFrame(dic1)
    return df

In [102]:
#1º paso: Inicializar el dataframe vacio
data_total = pd.DataFrame([], columns=['Marca', 'Modelo', 'Año', 'Potencia', 'Plazas', 'Nº de puertas',
         'km', 'Tipo de vehiculo', 'Combustible', 'Cambio', 'Comentarios','Nombre usuario',
          'Puntuacion', 'Nº valoraciones', 'Nº Ventas', 'Ubicacion', 'Precio' ])


In [104]:
#2º paso: Abrir el navegador de Selenium y quitar todos los Pop-up
navegador = webdriver.Chrome(service=driver_service)
navegador.get("https://es.wallapop.com/")

In [105]:
#3ºPaso establecer los parametros deseados 

#Cada iteración irá del precio minimo al precio minimo +1000, para abarcar el máximo número de coches posibles
# A partir de 4000 coches en num_coches el rendimiento del navegador empieza a ir lento (al menos el mio)
precio_minimo = 43000 ##Empezamos a buscar coches a partir del rango mínimo de precio (1000) no he puesto 0 porque salen ventas raras
precio_maximo = 44000 ##Precio máximo hasta donde queremos buscar. NO PUEDE SER IGUAL AL PRECIO MINIMO
num_coches = 3050 # Numero de coches que queremos coger por cada rango de precio. Es una medida APROXIMADA, siempre suele coger 100+ de lo indicado, irrelevante, queremos MUCHOS datos
links = [] ## Para la condicion del while
inicio = 0 ## Por que numero de link empezara a sacar datos. Por defecto el 0. 

for j in range((precio_maximo-precio_minimo)//1000):
    links = []
    contador = 0 ##Para que no se quede en bucle infinito si no encontrara los links, podría darse el caso de que no hubiera tal cantidad de links
    while len(links) < (num_coches-(num_coches/10)): ##Para cuando nos coge menos de 3000 links. Le dejo un margen de un 10% 3000 = 2700 - 3000        
        print("metemos coches de",precio_minimo)
        links = busca_coches(precio_minimo,num_coches) ## Sacame los datos de la pagina con el filtro aplicado, entre el rango de precios indicado arriba
        contador+=1 #Para romper el bucle en caso de que no encontrara el número de links que queremos. Le damos 3 intentos para buscarlo
        if contador > 3:
            break

    for j in range(inicio,len(links)+1):
        print("Link nº",j) #Así veriamos donde se ha dado el error
        dato_link = sacar_datos(links[j-1:j]) ## Que llame a la funcion tantas veces como numero de links tengamos
        data_total = pd.concat([data_total, dato_link], ignore_index=True) # No se como añadir el dataframe de dato_link sin concatenarlo
    
    data_total.to_csv(str(precio_minimo)+"CochesWallapop.csv",index = False)
    print("datos entre",str(precio_minimo),"y",str(precio_minimo+1000),"introducidos")
    precio_minimo+=1000 ## Pasamos al siguiente rango
    time.sleep(1) #Vamos a descansar
    
    data_total = pd.DataFrame([], columns=['Marca', 'Modelo', 'Año', 'Potencia', 'Plazas', 'Nº de puertas', #Reinicializamos el dataframe
         'km', 'Tipo de vehiculo', 'Combustible', 'Cambio', 'Comentarios','Nombre usuario',
          'Puntuacion', 'Nº valoraciones', 'Nº Ventas', 'Ubicacion', 'Precio' ])



metemos coches de 43000
coches de de 43000 a 44000
Cargando 0
Cargando 1
Cargando 2
Cargando 3
Cargando 4
Cargando 5
Cargando 6
Cargando 7
Cargando 8


KeyboardInterrupt: 